##### 0 - Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tqdm
import sklearn
import statsmodels as sm
import numba
from pathlib import Path

In [2]:
DIR_DATA = Path("..") / "data"
DIR_RESULTS = Path("..") / "results"
DIR_STOCK_DATA = DIR_DATA / "stock_data"

PATH_META = DIR_DATA / "metadata_listed.csv"
PATH_META_DELISTED = DIR_DATA / "metadata_delisted.csv"

In [3]:
df_meta = pd.read_csv(PATH_META)
df_target = df_meta.copy()

In [4]:
df_meta_delisted = pd.read_csv(PATH_META_DELISTED)
df_target_delisted = df_meta_delisted.copy()

In [5]:
all_stocks = []
p = 0.1

for row in tqdm.tqdm(df_target.itertuples(index=False), total=len(df_target), desc="Loading listed stocks"):
    x = np.random.binomial(1, p)
    if x == 0:
        continue
    
    symbol = row.Symbol
    path_stock = DIR_STOCK_DATA / f"{symbol}.csv"
    try:
        df_stock = pd.read_csv(path_stock)
    except Exception as e:
        continue

    df_stock.index = pd.MultiIndex.from_product([[symbol], df_stock.index], names=['ticker', 'date'])

    all_stocks.append(df_stock)

for row in tqdm.tqdm(df_target_delisted.itertuples(index=False), total=len(df_target_delisted), desc="Loading delisted stocks"):
    x = np.random.binomial(1, p)
    if x == 0:
        continue
    
    symbol = row.Symbol
    path_stock = DIR_STOCK_DATA / f"{symbol}.csv"
    try:
        df_stock = pd.read_csv(path_stock)
    except Exception as e:
        continue

    df_stock.index = pd.MultiIndex.from_product([[symbol], df_stock.index], names=['ticker', 'date'])

    all_stocks.append(df_stock)

df_stocks = pd.concat(all_stocks)

Loading delisted stocks: 100%|██████████| 9720/9720 [00:27<00:00, 349.72it/s]


In [6]:
df_stocks.sample(10)

,,ticker,date,open,high,low,close,adj_close,volume
ticker,date,,,,,,,,
MOGN,6276,MOGN,2007-04-26,22.710000,22.710000,22.710000,45.419070,45.419070,649066.0
TAYD,7858,TAYD,2016-10-07,19.980000,19.980000,19.950001,19.950001,19.950001,1200.0
TISI,5775,TISI,2003-08-22,4.000000,4.015000,3.950000,3.995000,3.995000,15600.0
EBAY,1126,EBAY,2003-03-20,9.219276,9.402357,9.165614,9.198495,9.198495,69426700.0
COLY,79,COLY,2005-12-01,16.240000,16.240000,16.240000,16.240056,16.240056,122357.0
DIT,3768,DIT,2010-07-23,57.250000,59.000000,57.250000,53.082115,53.082115,1300.0
EBAY,4438,EBAY,2016-05-16,23.680000,23.879999,23.610001,23.338818,23.338818,5589400.0
GL,5807,GL,2003-10-02,18.488890,18.520000,18.377777,14.088723,14.088723,709600.0
ABBV,3014,ABBV,2024-12-23,175.289993,178.649994,174.630005,172.363205,172.363205,6945600.0
